In [ ]:
# Notebook 1: Data Extraction & Cleaning

import os
import requests
import zipfile
import io
import getpass
import pandas as pd
from datetime import datetime

# Create directories if not exist
os.makedirs('../data/raw/origination', exist_ok=True)
os.makedirs('../data/raw/performance', exist_ok=True)
os.makedirs('../data/interim', exist_ok=True)

# --- 1. Automated login and download ---
login_url = "https://freddiemac.embs.com/FLoan/Data/download2.php"
download_url_template = "https://freddiemac.embs.com/FLoan/Data/sample_{year}.zip"

# username = input("Enter Freddie Mac portal username: ")
username = "bedant.lohanee@gmail.com"
# password = getpass.getpass("Enter password: ")
password = "GnyNpt<8"
credentials = {"username": username, "password": password}

session = requests.Session()
resp = session.post(login_url, data=credentials)
resp.raise_for_status()
if "Login Failed" in resp.text:
    raise Exception("Login failed — check your credentials")

for year in range(1999, 2000):
    url = download_url_template.format(year=year)
    print(f"Downloading {url}...")
    resp = session.get(url)
    if resp.status_code == 200:
        # Extract ZIP content
        z = zipfile.ZipFile(io.BytesIO(resp.content))
        for member in z.namelist():
            if member.endswith('.txt'):
                # Determine destination path based on file name
                if 'orig' in member.lower():
                    outpath = f"../data/raw/origination/sample_orig_{year}.txt"
                else:
                    outpath = f"../data/raw/performance/sample_perf_{year}.txt"
                print(f"  Extracting {member} to {outpath}")
                # Extract into a temp folder and then rename
                z.extract(member, path="../data/raw")
                os.rename(os.path.join("../data/raw", member), outpath)
    else:
        print(f"Failed to download year {year}: HTTP {resp.status_code}")
        # continue to next year if file not found

print("Download and extraction complete.")


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
  Extracting sample_orig_1999.txt to ../data/raw/origination/sample_orig_1999.txt
  Extracting sample_svcg_1999.txt to ../data/raw/performance/sample_perf_1999.txt
Download and extraction complete.
